In [ ]:
import osmnx as ox
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas as pd
from shapely.ops import transform
from functools import partial
import pyproj
import geojson
warnings.filterwarnings("ignore")


In [ ]:
osm_file = gpd.read_file('/home/idisc02/Desktop/i2sc/detect-car-LR/data/osm_json/berlin.geojson')

In [ ]:
osm_file["geometry"]

In [ ]:
print(osm_file["geometry"][6])

In [ ]:
osm_file['area'] = osm_file.geometry.area
osm_file['area']

In [ ]:
osm_file = osm_file.to_crs(pyproj.CRS.from_epsg(32633))
osm_file.crs

In [ ]:
osm_file['area'] = osm_file.geometry.area

In [ ]:
new_osm = osm_file[osm_file['area'] > 10000]

In [ ]:
new_osm = new_osm.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
osm_file = osm_file.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
new_osm.to_file('new_osm.geojson', driver='GeoJSON')

In [ ]:
print(osm_file.crs)
print(new_osm.crs)

In [ ]:
from shapely.geometry import Polygon
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the OSM geojson file
file_path = 'new_osm.geojson'
gdf = gpd.read_file(file_path)

# Iterate through each geometry and plot bounding box
for index, row in gdf.iterrows():
    geometry = row['geometry']

    if geometry.type == 'Polygon':
        polygons = [geometry]
    elif geometry.type == 'MultiPolygon':
        polygons = geometry.geoms
    else:
        continue
    
    for polygon in polygons:
        min_x, min_y, max_x, max_y = polygon.bounds
        fig, ax = plt.subplots()

        x, y = polygon.exterior.xy
        ax.plot(x, y, color='blue', label='Polygon')

        # Plot the bounding box
        bbox = plt.Rectangle((min_x, min_y), max_x - min_x, max_y - min_y,
                             fill=False, color='red', label='Bounding Box')
        ax.add_patch(bbox)

        ax.legend()

        plt.show()

        # Save the bounding box as a GeoJSON file
        bbox_geojson = gpd.GeoSeries([Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])])
        bbox_geojson.crs = gdf.crs  # Set the same coordinate reference system as the original data
        output_path = f'Berlin_Parking/parking_lot_{index}_bbox.geojson'
        bbox_geojson.to_file(output_path, driver='GeoJSON')